# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import os
import cv2
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
import glob
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

# Functions for extracting bounding boxes and loading dictionary

In [39]:
def extract_BBoxes(filename):
   #find root of file
    root = ET.parse(filename).getroot()
    
    boxes = list()
    names = list()
    # get bounding boxes and names
    for objct in root.findall(".//object"):
        name = objct.find('name').text
        xmin = int(objct.find('bndbox/xmin').text)
        ymin = int(objct.find('bndbox/ymin').text)
        xmax = int(objct.find('bndbox/xmax').text)
        ymax = int(objct.find('bndbox/ymax').text)
        names.append(name)
        boxes.append([xmin,ymin,xmax,ymax])

    return [boxes,names]
    
def load_dataset(path, deleteFiles = False):
    files_xml = [f for f in glob.glob(str(path) + "/*.xml")] # comes in random order
    
    imgbbox = dict()
    #print(len(files_xml))
    for file in files_xml:  
        
        imgFilePath = file[:-3] + "jpg"
        if os.path.exists(imgFilePath):  
            lbl_bbox = extract_BBoxes(file)  # Gets the bbox information
            
            imgbbox.update({imgFilePath.replace(str(path)+"/",''): lbl_bbox})
            
        elif deleteFiles:
            print("Found xml with no jpg")
            print("Deleting xml file: %s" %file)
            os.remove(file)
            print("Deleted")

    return imgbbox

# Loading images and making test/train data

In [108]:
imgbboxes = load_dataset('../Project2', deleteFiles=False)
new_imgbbox = {}
for img in imgbboxes:

    if len(imgbboxes[img][1]) == 1:
        new_imgbbox.update( {img: imgbboxes[img]})

labels = [new_imgbbox[file][1] for file in new_imgbbox]

path = '../Project2'

imgList = []
for file in new_imgbbox:
    temp = cv2.imread(os.path.join(path,file))
    if temp is not None:
        imgList.append(temp)

images = np.array(imgList)


n_categories = 2
split = 367

x_train = images[:split]
x_test = images[split:]

y_train = labels[:split]
for i in range(len(y_train)):
    if y_train[i][0] == 'handgun':
        y_train[i] = 1
        
    else:
        y_train[i] = 2

y_train = np.array(y_train)
y_train = keras.utils.to_categorical(y_train-1, n_categories)

y_test = labels[split:]
for i in range(len(y_test)):
    if y_test[i][0] == 'handgun':
        y_test[i] = 1
        
    else:
        y_test[i] = 2

y_test = np.array(y_test)
y_test = keras.utils.to_categorical(y_test-1, n_categories)

# Making model, training and evaluate

In [107]:
n, y, x, d = x_train.shape
inputShape = (y,x,d)
n_iterations = 5

model = Sequential()
model.add(layers.Conv2D(12, 3, activation='relu', input_shape=inputShape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(12, 3, activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(12, 3, activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(10, 3, activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(8, 3, activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(4, 3, activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(2, 3, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(units = 2, activation='softmax'))

# compiling model with cross entropy and accuracy
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

# training model
model.fit(x_train, y_train, epochs=n_iterations, verbose=1, validation_data=(x_test, y_test))

loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

print("\nAccuracy:\t", accuracy, "\nLoss:    \t", loss)

Epoch 1/5
12/12 [==============================] - 11s 897ms/step - loss: 1.4037 - accuracy: 0.6267 - val_loss: 0.7231 - val_accuracy: 0.6200
Epoch 2/5
12/12 [==============================] - 10s 864ms/step - loss: 0.6797 - accuracy: 0.6485 - val_loss: 0.7051 - val_accuracy: 0.6200
Epoch 3/5
12/12 [==============================] - 11s 945ms/step - loss: 0.6648 - accuracy: 0.6567 - val_loss: 0.7154 - val_accuracy: 0.6400
Epoch 4/5
12/12 [==============================] - 14s 1s/step - loss: 0.6685 - accuracy: 0.6485 - val_loss: 0.7018 - val_accuracy: 0.6200
Epoch 5/5
12/12 [==============================] - 11s 907ms/step - loss: 0.6524 - accuracy: 0.6567 - val_loss: 0.7069 - val_accuracy: 0.6300

Accuracy:	 0.6299999952316284 
Loss:    	 0.7069478034973145
